In [242]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing the dependencies

In [243]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
import time
import regex
import re
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score,precision_score,recall_score,f1_score,mean_absolute_error
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.svm import SVR

### importing the dataset

In [244]:
df = pd.read_csv("/content/drive/MyDrive/colab/uniacco/Data_Science_Internship - Dump.csv")

*** Checking the head values***

In [245]:
df.head(2)

,Unnamed: 0,Agent_id,status,lost_reason,budget,lease,movein,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id
0,0,1deba9e96f404694373de9749ddd1ca8aa7bb823145a6f...,LOST,Not responding,NaN,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,cd5dc0d9393f3980d11d4ba6f88f8110c2b7a7f7796307...
1,1,299ae77a4ef350ae0dd37d6bba1c002d03444fb1edb236...,LOST,Low budget,NaN,NaN,NaN,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6...,3d59f7548e1af2151b64135003ce63c0a484c26b9b8b16...,268ad70eb5bc4737a2ae28162cbca30118cc94520e49ef...,5372372f3bf5896820cb2819300c3e681820d82c6efc54...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,NaN,b94693673a5f7178d1b114e4004ad52377d3244dd24a3d...


**checking the datatypes**

In [246]:
df.dtypes

Unnamed: 0         int64
Agent_id          object
status            object
lost_reason       object
budget            object
lease             object
movein            object
source            object
source_city       object
source_country    object
utm_source        object
utm_medium        object
des_city          object
des_country       object
room_type         object
lead_id           object
dtype: object

# **Data Cleaning** and **Feature Selection**

**dropping the unnecessary columns**

In [247]:
df.drop(columns = "Unnamed: 0",inplace = True)

###### Checking the unique values in status column as we only need **WON** or **LOST** in our dataset

In [248]:
df['status'].unique()

array(['LOST', 'WON', 'OPPORTUNITY', 'CONTACTED', 'PROCESSING',
       'IMPORTANT'], dtype=object)

###### Removing the 'OPPORTUNITY', 'CONTACTED', 'PROCESSING', 'IMPORTANT' from dataset

In [249]:
df = df[df['status'] != "OPPORTUNITY"]
df = df[df['status'] != "CONTACTED"]
df = df[df['status'] != "PROCESSING"]
df = df[df['status'] != "IMPORTANT"]

###### Removing the nan values from our dataset

In [250]:
a = '9b2d5b4678781e53038e91ea5324530a03f27dc1d0e5f6c9bc9d493a23be9de0'            #### this is a nan value

In [251]:
cols = df.columns 
for i in cols:
    if len(df[df[i] == a]) > 0:
        df = df[df[i] != a] 

###### Replacing the nan ,inf ,-inf values with 0

In [252]:
a = df.columns
for i in a:
  df[i] = df[i].replace(np.nan, "0")

a = df.columns
for i in a:
  df[i] = df[i].replace(np.inf, "0")

a = df.columns
for i in a:
  df[i] = df[i].replace(-np.inf, "0")


df.replace(np.nan,"0")

,Agent_id,status,lost_reason,budget,lease,movein,source,source_city,source_country,utm_source,utm_medium,des_city,des_country,room_type,lead_id
2,c213697430c006013012dd2aca82dd9732aa0a1a6bca13...,LOST,Not responding,£121 - £180 Per Week,Full Year Course Stay 40 - 44 weeks,31/08/22,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,9b8cc3c63cdf447e463c11544924bf027945cbd29675f7...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,11ab03a1a8c367191355c152f39fe28cae5e426fce49ef...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Ensuite,96ea4e2bf04496c044745938c0299c264c3f4ba079e572...
3,eac9815a500f908736d303e23aa227f0957177b0e6756b...,LOST,Low budget,0-0,0,0,ba2d0a29556ac20f86f45e4543c0825428cba33fd7a9ea...,a5f0d2d08eb0592087e3a3a2f9c1ba2c67cc30f2efd2bd...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,19372fa44c57a01c37a5a8418779ca3d99b0b59731fb35...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,0,1d2b34d8add02a182a4129023766ca4585a8ddced0e5b3...
5,2306878a9ad9b57686cd623dd285aaa9b25afdf627f651...,LOST,Wants private accommodation,120,semester-stay,0,7bcfca0e9d73371699b0adbf1c691e02108fa64f02f4db...,7b7f15f9ffbeb905caaee265ac39d267d0512ab5333f2f...,8da82000ef9c4468ba47362a924b895e40662fed846942...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,f19d051291d4364989e4e957ef610448af3e05863ee1b8...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,0,b26f82b657d94e19ef8382877006c3e9425d7e4ab836f9...
6,2306878a9ad9b57686cd623dd285aaa9b25afdf627f651...,LOST,Short stay,0-0,0,0,9ba9134a91cfc6b52ac8d480e9ad37896ca4ac216e2d79...,3c1def48af45cb9748302f34e57f2cbeb866de95beb0d2...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,4688fa9ce4f64d0451be91f30a3de24a1c1c41635bd221...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,ecc0e7dc084f141b29479058967d0bc07dee25d9690a98...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,0,7ea9868bb6b4ebe610b0e509950b10322bcb7241d4a101...
7,44864c96fa1c36602f0d045b268981b6cab638a60fc207...,LOST,Wants private accommodation,£181 - £240 Per Week,51,08/09/22,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,8593b9db65212160979d946950601c26622a219c80f1f1...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,8593b9db65212160979d946950601c26622a219c80f1f1...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Entire Place,96ea4e2bf04496c044745938c0299c264c3f4ba079e572...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46599,eac9815a500f908736d303e23aa227f0957177b0e6756b...,LOST,Low availability,£60 - £120 Per week,Semester Stay 20 - 24 weeks,31/08/22,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,42e535be8863a0253115744dca757cddddfc2f626df4f8...,e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,085c3a49a31cbfa71c36746a98dc908d30d423a63b969c...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Entire Place,86ae1d25e92121dd581c75c958103bb3cc81ef4aa47fef...
46602,d119aab7bc5c0823d29df1fa368bc56a6f6e333fc680a2...,LOST,Wants private accommodation,£121 - £180 Per Week,Other,19/09/22,7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2...,7b7f15f9ffbeb905caaee265ac39d267d0512ab5333f2f...,8da82000ef9c4468ba47362a924b895e40662fed846942...,bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07...,09076eb7665d1fb9389c7c4517fee0b00e43092eb34821...,6a5ed83658ef85afc77709cc16ede854ad98e3c2ad8b07...,8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512...,Entire Place,d9e0f455b68a6573544ffac32292541a510fe2daffe2bb...
46605,1134c0a7d44fdae1afd7f1f64e2789496784095ca0a050...,LOST,Low availability,£241 - £300 Per week,Full Year Course Stay 40 - 44 weeks,20/09/22,7aae3e886e89fc1187a5c4

###### Finding the average budget from budget column

In [253]:
avgr= []
for i in range(len(df['budget'])):
    strs = df['budget'].iloc[i]
    x = re.findall('[0-9]+', strs)
    n = len(x)
    a = []
    for i in range(n):
        a.append(int(x[i]))
    try:
        avgr.append(sum(a)/len(a))   
    except:
        avgr.append(0)

###### Placing the avgr budget list into the dataframe

In [254]:
df['budget_new'] = avgr

##### Finding the average span in week 

In [255]:
span_avg = []
for i in range(len(df['lease'])):

    if "week" in df['lease'].iloc[i] or "weekks" in  "weeeks" in df['lease'].iloc[i] or df['lease'].iloc[i] or "WEEK" in df['lease'].iloc[i] or "weeks" in df['lease'].iloc[i]:
        strs = df['lease'].iloc[i]
        x = re.findall('[0-9]+', strs)
        n = len(x)
        a = []
        for i in range(n):
            a.append(int(x[i]))
        try:
            span_avg.append(sum(a)/len(a))   
        except:
            span_avg.append(0)
    elif "Other" in df['lease'].iloc[i] or "not sure" in df['lease'].iloc[i]:
        span_avg.append("NA")
    elif  "Full Year" in df['lease'].iloc[i] or  "full year stay" in df['lease'].iloc[i] or "year" in df['lease'].iloc[i] or "full year" in df['lease'].iloc[i] or "Full year" in df['lease'].iloc[i]:
        span_avg.append(52)
    elif "month" in df['lease'].iloc[i] or "months" in df['lease'].iloc[i]:
          strs = df['lease'].iloc[i]
          x = re.findall('[0-9]+', strs)
          n = len(x)
          a = []
          for i in range(n):
              a.append(4*int(x[i]))
          try:
              span_avg.append(sum(a)/len(a))   
          except:
              span_avg.append(0)
    elif  "short stay" in df['lease'].iloc[i] or "Short Stay" in df['lease'].iloc[i]:
        span_avg.append(10)
    elif "Full year stay" in df['lease'].iloc[i] or "Full Year Stay" in df['lease'].iloc[i] or "Academic year" in df['lease'].iloc[i] or "long tearm" in df['lease'].iloc[i]:
        span_avg.append(52) 
    elif "aug" in df['lease'].iloc[i]:
      span_avg.append(4)
    elif "years" in df['lease'].iloc[i]:
          strs = df['lease'].iloc[i]
          x = re.findall('[0-9]+', strs)
          n = len(x)
          a = []
          for i in range(n):
              a.append(52*int(x[i]))
          try:
              span_avg.append(sum(a)/len(a))   
          except:
              span_avg.append(0)
    elif "days" in df['lease'].iloc[i]:
        strs = df['lease'].iloc[i]
        x = re.findall('[0-9]+', strs)
        n = len(x)
        a = []
        for i in range(n):
            a.append(int(x[i])/7)
        try:
            span_avg.append(sum(a)/len(a))   
        except:
            span_avg.append(0)

    elif "Sem stay" in df['lease'].iloc[i]:
        span_avg.append(22)
    else:
        strs = df['lease'].iloc[i]
        x = re.findall('[0-9]+', strs)
        n = len(x)
        a = []
        for i in range(n):
            a.append(int(x[i]))
        try:
            span_avg.append(sum(a)/len(a))   
        except:
            span_avg.append(0)

###### Placing the span_avg week list into the dataframe

In [256]:
df['lease_span_avg_week'] = span_avg

### LabelEncoding the categorical columns

In [257]:
label_encoder = preprocessing.LabelEncoder()
df['Agent_id_encoded']= label_encoder.fit_transform(df['Agent_id'])
dic_Agent_id = {}
for i in df['Agent_id'].unique():
    if i not in dic_Agent_id:
      dic_Agent_id[i]  = df[df['Agent_id']==i]['Agent_id_encoded'].iloc[0]

In [258]:
label_encoder = preprocessing.LabelEncoder()
df['status_encoded']= label_encoder.fit_transform(df['status'])
dic_status = {}
for i in df['status'].unique():
    if i not in dic_status:
      dic_status[i]  = df[df['status']==i]['status_encoded'].iloc[0]

In [259]:
label_encoder = preprocessing.LabelEncoder()
df['lost_reason_encoded']= label_encoder.fit_transform(df['lost_reason'])
dic_lost_reason = {}
for i in df['lost_reason'].unique():
    if i not in dic_lost_reason:
      dic_lost_reason[i]  = df[df['lost_reason']==i]['lost_reason_encoded'].iloc[0]      

In [260]:
label_encoder = preprocessing.LabelEncoder()
df['source_encoded']= label_encoder.fit_transform(df['source'])
dic_source = {}
for i in df['source'].unique():
    if i not in dic_source:
      dic_source[i]  = df[df['source']==i]['source_encoded'].iloc[0]

In [261]:
label_encoder = preprocessing.LabelEncoder()
df['source_city_encoded']= label_encoder.fit_transform(df['source_city'])
dic_source_city = {}
for i in df['source_city'].unique():
    if i not in dic_source_city:
      dic_source_city[i]  = df[df['source_city']==i]['source_city_encoded'].iloc[0]

In [262]:
label_encoder = preprocessing.LabelEncoder()
df['source_country_encoded']= label_encoder.fit_transform(df['source_country'])
dic_source_country = {}
for i in df['source_country'].unique():
    if i not in dic_source_country:
      dic_source_country[i]  = df[df['source_country']==i]['source_country_encoded'].iloc[0]

In [263]:
label_encoder = preprocessing.LabelEncoder()
df['utm_source_encoded']= label_encoder.fit_transform(df['utm_source'])
dic_utm_source = {}
for i in df['utm_source'].unique():
    if i not in dic_utm_source:
      dic_utm_source[i]  = df[df['utm_source']==i]['utm_source_encoded'].iloc[0]

In [264]:
label_encoder = preprocessing.LabelEncoder()
df['utm_medium_encoded']= label_encoder.fit_transform(df['utm_medium'])
dic_utm_medium = {}
for i in df['utm_medium'].unique():
    if i not in dic_utm_medium:
      dic_utm_medium[i]  = df[df['utm_medium']==i]['utm_medium_encoded'].iloc[0]

In [265]:
label_encoder = preprocessing.LabelEncoder()
df['des_city_encoded']= label_encoder.fit_transform(df['des_city'])
dic_des_city = {}
for i in df['des_city'].unique():
    if i not in dic_des_city:
      dic_des_city[i]  = df[df['des_city']==i]['des_city_encoded'].iloc[0]

In [266]:
label_encoder = preprocessing.LabelEncoder()
df['des_country_encoded']= label_encoder.fit_transform(df['des_country'])
dic_des_country = {}
for i in df['des_country'].unique():
    if i not in dic_des_country:
      dic_des_country[i]  = df[df['des_country']==i]['des_country_encoded'].iloc[0]

In [267]:
label_encoder = preprocessing.LabelEncoder()
df['room_type_encoded']= label_encoder.fit_transform(df['room_type'])
dic_room_type = {}
for i in df['room_type'].unique():
    if i not in dic_room_type:
      dic_room_type[i]  = df[df['room_type']==i]['room_type_encoded'].iloc[0]

Creating a new datafram as **acc_df** from df

In [268]:
acc_df = df[['budget_new',
       'lease_span_avg_week', 'Agent_id_encoded', 'status_encoded',
       'lost_reason_encoded', 'source_encoded', 'source_city_encoded',
       'source_country_encoded', 'utm_source_encoded', 'utm_medium_encoded',
       'des_city_encoded', 'des_country_encoded', 'room_type_encoded',]]

###### Reset the index and drop the unnamed: 0 columns from acc_df

In [269]:
acc_df.reset_index(inplace = True)

In [270]:
acc_df.drop(columns = "index",inplace = True)

### Outlier Handling

In [271]:
def outlier_handler(X):
    for i in X.columns:    
        q1 = np.percentile(X[i],25,interpolation  = 'midpoint')
        q3 = np.percentile(X[i],75,interpolation  = 'midpoint')
        IQR = q3-q1
        # Upper bound
        upper = np.where(X[i] >= (q3+1.5*IQR))
        # Lower bound
        lower = np.where(X[i] <= (q1-1.5*IQR))
        try:
            X.drop(upper[0], inplace = True)
            X.drop(lower[0], inplace = True)
        except:pass 

In [272]:
outlier_handler(acc_df)

##### Converting the dataframe columns  to float64 type

In [273]:
acc_df["budget_new"] = acc_df["budget_new"].astype("float64")
acc_df["lease_span_avg_week"] = acc_df["lease_span_avg_week"].astype("float64")
acc_df["lost_reason_encoded"] = acc_df["lost_reason_encoded"].astype("float64")
acc_df["Agent_id_encoded"] = acc_df["Agent_id_encoded"].astype("float64")
acc_df["status_encoded"] = acc_df["status_encoded"].astype("float64")
acc_df["source_encoded"] = acc_df["source_encoded"].astype("float64")
acc_df["source_city_encoded"] = acc_df["source_city_encoded"].astype("float64")
acc_df["utm_source_encoded"] = acc_df["utm_source_encoded"].astype("float64")
acc_df["utm_medium_encoded"] = acc_df["utm_medium_encoded"].astype("float64")
acc_df["des_city_encoded"] = acc_df["des_city_encoded"].astype("float64")
acc_df["des_country_encoded"] = acc_df["des_country_encoded"].astype("float64")
acc_df["room_type_encoded"] = acc_df["room_type_encoded"].astype("float64")
acc_df["source_country_encoded"] = acc_df["source_country_encoded"].astype("float64")

In [274]:
acc_df.dtypes

budget_new                float64
lease_span_avg_week       float64
Agent_id_encoded          float64
status_encoded            float64
lost_reason_encoded       float64
source_encoded            float64
source_city_encoded       float64
source_country_encoded    float64
utm_source_encoded        float64
utm_medium_encoded        float64
des_city_encoded          float64
des_country_encoded       float64
room_type_encoded         float64
dtype: object

#### Saving the cleaned dataset

In [275]:
#acc_df.to_csv("new_uniaaco.csv")

In [276]:
#df = pd.read_csv("/content/drive/MyDrive/colab/uniacco/new_uniacco.csv")

# **Model Building** 

### Dividing the acc_df into X and Y dataframes

In [277]:
#acc_df = df

In [278]:
#acc_df.drop(columns = "Unnamed: 0",inplace = True)

In [279]:
X = acc_df[['budget_new', 'lease_span_avg_week', 'Agent_id_encoded',
            'lost_reason_encoded', 'source_encoded',
       'source_city_encoded', 'source_country_encoded', 'utm_source_encoded',
       'utm_medium_encoded', 'des_city_encoded', 'des_country_encoded',
       'room_type_encoded']]
Y = acc_df['status_encoded']

In [280]:
X

,budget_new,lease_span_avg_week,Agent_id_encoded,lost_reason_encoded,source_encoded,source_city_encoded,source_country_encoded,utm_source_encoded,utm_medium_encoded,des_city_encoded,des_country_encoded,room_type_encoded
0,150.5,42.0,80.0,21.0,98.0,2345.0,160.0,19.0,2.0,17.0,8.0,1.0
1,0.0,0.0,103.0,16.0,146.0,2522.0,160.0,19.0,2.0,25.0,8.0,0.0
2,120.0,0.0,13.0,29.0,99.0,1877.0,98.0,19.0,2.0,195.0,8.0,0.0
3,0.0,0.0,13.0,25.0,126.0,924.0,160.0,8.0,2.0,190.0,8.0,0.0
4,210.5,51.0,25.0,29.0,98.0,2033.0,160.0,19.0,2.0,109.0,8.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
30869,150.5,42.0,53.0,15.0,98.0,2695.0,160.0,19.0,2.0,190.0,8.0,1.0
30870,90.0,42.0,37.0,20.0,98.0,3572.0,160.0,19.0,2.0,33.0,8.0,2.0
30871,90.0,22.0,103.0,15.0,98.0,1022.0,160.0,19.0,2.0,6.0,8.0,2.0
30872,150.5,0.0,90.0,29.0,98.0,1877.0,98.0,19.0,2.0,85.0,8.0,2.0


#### Creating the dictionary that will contain all the scores of the models

In [281]:
final_dict = {"Feature":[],"Score":[],"Model_name":[],"rmse_val":[],"r2_score":[],"mse":[],"mae":[],"f1":[],"precision":[],'recall':[]}
def add_record(df,feature,model,rmse,r2,mse,mae,f1,precision,recall):
    df['Feature'].append(feature)
    df["Score"].append(score)
    df["Model_name"].append(model)
    df['rmse_val'].append(rmse)
    df["r2_score"].append(r2)
    df["mse"].append(mse)
    df['mae'].append(mae)
    df['f1'].append(f1)
    df['precision'].append(precision)
    df['recall'].append(recall)

### Perform the train Test split dataset

In [282]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size =0.3,random_state = 42)

##### Training the linear regression model 

In [283]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

##### Predictions

In [284]:
predicted = model.predict(x_test)
predicted

array([ 0.01111536,  0.0199548 , -0.02719723, ...,  0.45382743,
        0.09172641,  0.05158683])

##### Score

In [285]:
mse = mean_squared_error(y_test,predicted)
r2 = r2_score(y_test,predicted)
#f1 = f1_score(y_test,predicted)
#precision = precision_score(y_test,predicted)
print(f"Mean Squared Error : {mse}")
print(f"r2_score : {r2}")
#print(f"f1 score : {f1}")
#print(f"precision score : {precision}")

Mean Squared Error : 0.04077903936180981
r2_score : 0.4235078511823416


## Building a predictive system

In [286]:
def predict(a):
    a.fit(x_train,y_train)
    budget_new = float(input("Enter the budget_new value: "))
    lease_span_avg_week = float(input("Enter the lease_span_avg_week value: "))
    lost_reason = input("Enter the lost_reason value: ")
    lost_reason = dic_lost_reason[lost_reason]
    Agent_id =  input("Enter the Agent_id value: ")
    Agent_id = dic_Agent_id[Agent_id]
    source =  input("Enter the source value: ")
    source = dic_source[source]
    source_city =  input("Enter the source_city value: ")
    source_city = dic_source_city[source_city]
    source_country =  input("Enter the source_country value: ")
    source_country = dic_source_country[source_country]
    utm_source =  input("Enter the utm_source value: ")
    utm_source = dic_utm_source[utm_source]
    utm_medium =  input("Enter the utm_medium value: ")
    utm_medium = dic_utm_medium[utm_medium]
    des_city =  input("Enter the des_city value: ")
    des_city = dic_des_city[des_city]
    des_country =  input("Enter the des_country value: ")
    des_country = dic_des_country[des_country]
    room_type =  input("Enter the room_type value: ")
    room_type = dic_room_type[room_type]

    return a.predict([[budget_new,lease_span_avg_week,lost_reason,Agent_id,source,source_city,source_country,utm_source,
                      utm_medium, des_city,des_country,room_type]])

In [ ]:
predict(model)

Enter the budget_new value: 1521
Enter the lease_span_avg_week value: 142
Enter the lost_reason value: Not responding
Enter the Agent_id value: c213697430c006013012dd2aca82dd9732aa0a1a6bca1371c92f18fd11d9bc5f
Enter the source value: 7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2b7c51be879f0c37ca8
Enter the source_city value: 9b8cc3c63cdf447e463c11544924bf027945cbd29675f77955bb36364356c14e
Enter the source_country value: e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a9750aa698e797e9c96
Enter the utm_source value: bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07ee6acc68762d12c2e3
Enter the utm_medium value: 09076eb7665d1fb9389c7c4517fee0b00e43092eb34821b09b5730c41ebcc50c
Enter the des_city value: 11ab03a1a8c367191355c152f39fe28cae5e426fce49efb320230ca4ae3f97a1
Enter the des_country value: 8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512ff098eaa56a4d5fb19
Enter the room_type value: Ensuite


array([-1.2476742])

#  I have built different models , all of them are below

## Simple Linear Regression

In [287]:
new_y = acc_df['status_encoded']

In [288]:
for i in X.columns:
    new_x = X[[i]]
    new_y = acc_df['status_encoded']
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    model = LinearRegression()
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    
    score = [model.score(x_train,y_train),model.score(x_test,y_test)]
    mse = mean_squared_error(y_test,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test,y_pred))
    r2 = r2_score(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i,"Simple_Linear",rmse,r2,mse,mae,f1,precision,recall)

In [289]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,0.265456,0.003813,0.070467,0.137974,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,0.263959,0.015019,0.069674,0.136337,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,0.265966,-0.000017,0.070738,0.138310,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,0.203131,0.416675,0.041262,0.132742,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,0.265754,0.001572,0.070625,0.138332,na,na,na
5,source_city_encoded,"[4.479857665329323e-05, 9.514095727403138e-05]",Simple_Linear,0.265951,0.000095,0.070730,0.138416,na,na,na
6,source_country_encoded,"[0.0016500185303758297, 0.0007092025128553336]",Simple_Linear,0.265869,0.000709,0.070686,0.138216,na,na,na
7,utm_source_encoded,"[4.633342586268796e-06, -0.00018931905736763177]",Simple_Linear,0.265989,-0.000189,0.070750,0.138478,na,na,na
8,utm_medium_encoded,"[0.0018746669142841732, 0.000343827167472055]",Simple_Linear,0.265918,0.000344,0.070712,0.138193,na,na,na
9,des_city_encoded,"[0.00023326293621683813, -7.071305983541443e-05]",Simple_Linear,0.265973,-0.000071,0.070742,0.138422,na,na,na


# Multiple Linear Regression

In [290]:
for i in X.columns:
    
    new = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded']]
    if i in new.columns:
        new_x = X.drop(i,axis = 1)
    new_y =  acc_df['status_encoded']  
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    model = LinearRegression()
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    
    score = [model.score(x_train,y_train),model.score(x_test,y_test)]
    mse = mean_squared_error(y_test,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test,y_pred))
    r2 = r2_score(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i,"Multiple Regression",rmse,r2,mse,mae,f1,precision,recall)

In [291]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,0.265456,0.003813,0.070467,0.137974,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,0.263959,0.015019,0.069674,0.136337,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,0.265966,-0.000017,0.070738,0.138310,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,0.203131,0.416675,0.041262,0.132742,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,0.265754,0.001572,0.070625,0.138332,na,na,na
5,source_city_encoded,"[4.479857665329323e-05, 9.514095727403138e-05]",Simple_Linear,0.265951,0.000095,0.070730,0.138416,na,na,na
6,source_country_encoded,"[0.0016500185303758297, 0.0007092025128553336]",Simple_Linear,0.265869,0.000709,0.070686,0.138216,na,na,na
7,utm_source_encoded,"[4.633342586268796e-06, -0.00018931905736763177]",Simple_Linear,0.265989,-0.000189,0.070750,0.138478,na,na,na
8,utm_medium_encoded,"[0.0018746669142841732, 0.000343827167472055]",Simple_Linear,0.265918,0.000344,0.070712,0.138193,na,na,na
9,des_city_encoded,"[0.00023326293621683813, -7.071305983541443e-05]",Simple_Linear,0.265973,-0.000071,0.070742,0.138422,na,na,na


In [292]:
a = ['budget_new', 'lease_span_avg_week', 'Agent_id_encoded',
     'lost_reason_encoded', 'source_encoded', 'source_city_encoded',
     'des_city_encoded', 'des_country_encoded', 'room_type_encoded']

# LASSO

In [293]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    #print(x_train.shape)
    #print(pd.DataFrame(y_train).shape)
    #break
    # Standarizing the dataset
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = Lasso(alpha = 0.1)
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i,"Lasso Regression",rmse,r2,mse,mae,f1,precision,recall)

In [294]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,0.265456,0.003813,0.070467,0.137974,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,0.263959,0.015019,0.069674,0.136337,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,0.265966,-0.000017,0.070738,0.138310,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,0.203131,0.416675,0.041262,0.132742,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,0.265754,0.001572,0.070625,0.138332,na,na,na
5,source_city_encoded,"[4.479857665329323e-05, 9.514095727403138e-05]",Simple_Linear,0.265951,0.000095,0.070730,0.138416,na,na,na
6,source_country_encoded,"[0.0016500185303758297, 0.0007092025128553336]",Simple_Linear,0.265869,0.000709,0.070686,0.138216,na,na,na
7,utm_source_encoded,"[4.633342586268796e-06, -0.00018931905736763177]",Simple_Linear,0.265989,-0.000189,0.070750,0.138478,na,na,na
8,utm_medium_encoded,"[0.0018746669142841732, 0.000343827167472055]",Simple_Linear,0.265918,0.000344,0.070712,0.138193,na,na,na
9,des_city_encoded,"[0.00023326293621683813, -7.071305983541443e-05]",Simple_Linear,0.265973,-0.000071,0.070742,0.138422,na,na,na


# Ridge Regression

In [295]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    # Standarizing the dataset
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    
    # Model training 
    model = Ridge(alpha = 0.1)
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i,"Ridge Regression",rmse,r2,mse,mae,f1,precision,recall)

# Elastic Net

In [296]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = ElasticNet(alpha = 0.1)
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i,"ElasticNet Regression",rmse,r2,mse,mae,f1,precision,recall)

In [297]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,0.265456,0.003813,0.070467,0.137974,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,0.263959,0.015019,0.069674,0.136337,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,0.265966,-0.000017,0.070738,0.138310,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,0.203131,0.416675,0.041262,0.132742,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,0.265754,0.001572,0.070625,0.138332,na,na,na
5,source_city_encoded,"[4.479857665329323e-05, 9.514095727403138e-05]",Simple_Linear,0.265951,0.000095,0.070730,0.138416,na,na,na
6,source_country_encoded,"[0.0016500185303758297, 0.0007092025128553336]",Simple_Linear,0.265869,0.000709,0.070686,0.138216,na,na,na
7,utm_source_encoded,"[4.633342586268796e-06, -0.00018931905736763177]",Simple_Linear,0.265989,-0.000189,0.070750,0.138478,na,na,na
8,utm_medium_encoded,"[0.0018746669142841732, 0.000343827167472055]",Simple_Linear,0.265918,0.000344,0.070712,0.138193,na,na,na
9,des_city_encoded,"[0.00023326293621683813, -7.071305983541443e-05]",Simple_Linear,0.265973,-0.000071,0.070742,0.138422,na,na,na


# Using Decision Tree Model

In [298]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = DecisionTreeRegressor(ccp_alpha = 0.1)
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i,"DecisionTree Regression",rmse,r2,mse,mae,f1,precision,recall)

In [299]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,2.654557e-01,0.003813,7.046675e-02,1.379744e-01,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,2.639585e-01,0.015019,6.967411e-02,1.363366e-01,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,2.659656e-01,-0.000017,7.073772e-02,1.383098e-01,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,2.031314e-01,0.416675,4.126236e-02,1.327424e-01,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,2.657542e-01,0.001572,7.062532e-02,1.383324e-01,na,na,na
5,source_city_encoded,"[4.479857665329323e-05, 9.514095727403138e-05]",Simple_Linear,2.659507e-01,0.000095,7.072977e-02,1.384160e-01,na,na,na
6,source_country_encoded,"[0.0016500185303758297, 0.0007092025128553336]",Simple_Linear,2.658690e-01,0.000709,7.068634e-02,1.382160e-01,na,na,na
7,utm_source_encoded,"[4.633342586268796e-06, -0.00018931905736763177]",Simple_Linear,2.659885e-01,-0.000189,7.074990e-02,1.384776e-01,na,na,na
8,utm_medium_encoded,"[0.0018746669142841732, 0.000343827167472055]",Simple_Linear,2.659176e-01,0.000344,7.071218e-02,1.381932e-01,na,na,na
9,des_city_encoded,"[0.00023326293621683813, -7.071305983541443e-05]",Simple_Linear,2.659728e-01,-0.000071,7.074151e-02,1.384219e-01,na,na,na


In [300]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    
    #scaler_X = StandardScaler()
    #scaler_Y = StandardScaler()
    #x_train_scaled = scaler_X.fit_transform(x_train)
    #y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    #x_test_scaled = scaler_X.transform(x_test)
    #y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = DecisionTreeClassifier(ccp_alpha = 0.1)
    model.fit(x_train,y_train)
    y_pred = model.predict((x_test))
    
    
    # Finding the score 
    score = [model.score(x_train,y_train),model.score(x_test,y_test)]
    mse = mean_squared_error(y_test,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test,y_pred))
    r2 = r2_score(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)
    add_record(final_dict,i,"DecisionTree Classification",rmse,r2,mse,mae,f1,precision,recall)

In [301]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,0.265456,0.003813,0.070467,0.137974,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,0.263959,0.015019,0.069674,0.136337,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,0.265966,-0.000017,0.070738,0.138310,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,0.203131,0.416675,0.041262,0.132742,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,0.265754,0.001572,0.070625,0.138332,na,na,na
...,...,...,...,...,...,...,...,...,...,...
64,source_encoded,"[1.0, 1.0]",DecisionTree Classification,0.000000,1.000000,0.000000,0.000000,1.0,1.0,1.0
65,source_city_encoded,"[1.0, 1.0]",DecisionTree Classification,0.000000,1.000000,0.000000,0.000000,1.0,1.0,1.0
66,des_city_encoded,"[1.0, 1.0]",DecisionTree Classification,0.000000,1.000000,0.000000,0.000000,1.0,1.0,1.0
67,des_country_encoded,"[1.0, 1.0]",DecisionTree Classification,0.000000,1.000000,0.000000,0.000000,1.0,1.0,1.0


# RandomForestRegressor

In [302]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = RandomForestRegressor(ccp_alpha = 0.1)
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i," RandomForest Regression",rmse,r2,mse,mae,f1,precision,recall)

In [303]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,2.654557e-01,0.003813,7.046675e-02,1.379744e-01,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,2.639585e-01,0.015019,6.967411e-02,1.363366e-01,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,2.659656e-01,-0.000017,7.073772e-02,1.383098e-01,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,2.031314e-01,0.416675,4.126236e-02,1.327424e-01,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,2.657542e-01,0.001572,7.062532e-02,1.383324e-01,na,na,na
...,...,...,...,...,...,...,...,...,...,...
73,source_encoded,"[1.0, 1.0]",RandomForest Regression,2.381864e-14,1.000000,5.673277e-28,2.367648e-14,na,na,na
74,source_city_encoded,"[1.0, 1.0]",RandomForest Regression,2.388420e-14,1.000000,5.704550e-28,2.377854e-14,na,na,na
75,des_city_encoded,"[1.0, 1.0]",RandomForest Regression,2.381864e-14,1.000000,5.673277e-28,2.367648e-14,na,na,na
76,des_country_encoded,"[1.0, 1.0]",RandomForest Regression,2.380927e-14,1.000000,5.668812e-28,2.369326e-14,na,na,na


In [304]:
for i in a:
    
    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
    
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    """
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    """
    
    # Model training 
    model = RandomForestClassifier(ccp_alpha = 0.1)
    model.fit(x_train,y_train)
    y_pred = model.predict((x_test))
    
    
    # Finding the score 
    score = [model.score(x_train,y_train),model.score(x_test,y_test)]
    mse = mean_squared_error(y_test,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test,y_pred))
    r2 = r2_score(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)
    add_record(final_dict,i,"RandomForest Classification",rmse,r2,mse,mae,f1,precision,recall)

In [305]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,0.265456,0.003813,0.070467,0.137974,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,0.263959,0.015019,0.069674,0.136337,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,0.265966,-0.000017,0.070738,0.138310,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,0.203131,0.416675,0.041262,0.132742,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,0.265754,0.001572,0.070625,0.138332,na,na,na
...,...,...,...,...,...,...,...,...,...,...
82,source_encoded,"[0.9269318469723841, 0.9233952003782953]",RandomForest Classification,0.276776,-0.082960,0.076605,0.076605,0.0,0.0,0.0
83,source_city_encoded,"[0.9269318469723841, 0.9233952003782953]",RandomForest Classification,0.276776,-0.082960,0.076605,0.076605,0.0,0.0,0.0
84,des_city_encoded,"[1.0, 1.0]",RandomForest Classification,0.000000,1.000000,0.000000,0.000000,1.0,1.0,1.0
85,des_country_encoded,"[0.9269318469723841, 0.9233952003782953]",RandomForest Classification,0.276776,-0.082960,0.076605,0.076605,0.0,0.0,0.0


# using XGBRegressor

In [306]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = XGBRegressor()
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i," XGBRegressor",rmse,r2,mse,mae,f1,precision,recall)

In [307]:
pd.DataFrame(final_dict)

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,0.265456,0.003813,7.046675e-02,0.137974,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,0.263959,0.015019,6.967411e-02,0.136337,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,0.265966,-0.000017,7.073772e-02,0.138310,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,0.203131,0.416675,4.126236e-02,0.132742,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,0.265754,0.001572,7.062532e-02,0.138332,na,na,na
...,...,...,...,...,...,...,...,...,...,...
91,source_encoded,"[0.9999999999714506, 0.9999999999714537]",XGBRegressor,0.000005,1.000000,2.981379e-11,0.000003,na,na,na
92,source_city_encoded,"[0.9999999999714506, 0.9999999999714537]",XGBRegressor,0.000005,1.000000,2.981379e-11,0.000003,na,na,na
93,des_city_encoded,"[0.9999999999714506, 0.9999999999714537]",XGBRegressor,0.000005,1.000000,2.981379e-11,0.000003,na,na,na
94,des_country_encoded,"[0.9999999999714506, 0.9999999999714537]",XGBRegressor,0.000005,1.000000,2.981379e-11,0.000003,na,na,na


# using GradientBoostingRegressor

In [308]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = GradientBoostingRegressor()
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i," GradientBoostingRegressor",rmse,r2,mse,mae,f1,precision,recall)

# Using KNN Regressor

In [309]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = KNeighborsRegressor()
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i," KNeighborsRegressor",rmse,r2,mse,mae,f1,precision,recall)

# using SVM Regressor

In [310]:
for i in a:

    new_x = X[['budget_new','lease_span_avg_week','lost_reason_encoded','source_encoded',i]]
 
    # Perform the train test split
    
    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)
    
    
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = SVR()
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))
    
    
    # Finding the score 
    score = [model.score(x_train_scaled,y_train_scaled),model.score(x_test_scaled,y_test_scaled)]
    mse = mean_squared_error(y_test_scaled,y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_scaled,y_pred))
    r2 = r2_score(y_test_scaled,y_pred)
    mae = mean_absolute_error(y_test_scaled,y_pred)
    precision = "na"
    recall = "na"
    f1 = "na"
    add_record(final_dict,i,"SVM",rmse,r2,mse,mae,f1,precision,recall)

#  Building the final Predictive system

In [311]:
def predict(model):
    new_x = X[['budget_new', 'lease_span_avg_week', 'Agent_id_encoded',
       'lost_reason_encoded', 'source_encoded', 'source_city_encoded',
       'source_country_encoded', 'utm_source_encoded', 'utm_medium_encoded',
       'des_city_encoded', 'des_country_encoded', 'room_type_encoded']]
    new_y = acc_df['status_encoded']

    # Perform the train test split

    x_train,x_test,y_train,y_test = train_test_split(new_x,new_y,test_size = 0.3,random_state = 42)

    # Standarizing the dataset

    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    x_train_scaled = scaler_X.fit_transform(x_train)
    y_train_scaled = scaler_Y.fit_transform(pd.DataFrame(y_train))
    x_test_scaled = scaler_X.transform(x_test)
    y_test_scaled = scaler_Y.transform(pd.DataFrame(y_test))
    
    # Model training 
    model = model()
    model.fit(x_train_scaled,y_train_scaled)
    y_pred = model.predict((x_test_scaled))


    budget_new = float(input("Enter the budget_new value: "))
    lease_span_avg_week = float(input("Enter the lease_span_avg_week value: "))
    lost_reason = input("Enter the lost_reason value: ")
    lost_reason = dic_lost_reason[lost_reason]
    Agent_id =  input("Enter the Agent_id value: ")
    Agent_id = dic_Agent_id[Agent_id]
    source =  input("Enter the source value: ")
    source = dic_source[source]
    source_city =  input("Enter the source_city value: ")
    source_city = dic_source_city[source_city]
    source_country =  input("Enter the source_country value: ")
    source_country = dic_source_country[source_country]
    utm_source =  input("Enter the utm_source value: ")
    utm_source = dic_utm_source[utm_source]
    utm_medium =  input("Enter the utm_medium value: ")
    utm_medium = dic_utm_medium[utm_medium]
    des_city =  input("Enter the des_city value: ")
    des_city = dic_des_city[des_city]
    des_country =  input("Enter the des_country value: ")
    des_country = dic_des_country[des_country]
    room_type =  input("Enter the room_type value: ")
    room_type = dic_room_type[room_type]

    print("the expected lead score is :",abs(100 * model.predict([[budget_new,lease_span_avg_week,lost_reason,Agent_id,source,source_city,source_country,utm_source,
                      utm_medium, des_city,des_country,room_type]])))
    a = abs(100 * model.predict([[budget_new,lease_span_avg_week,lost_reason,Agent_id,source,source_city,source_country,utm_source,
                      utm_medium, des_city,des_country,room_type]]))
    if a>25:
      print("We won customer")
    else:
      print("we lost customer")

In [ ]:
predict(SVR)

Enter the budget_new value: 151
Enter the lease_span_avg_week value: 52
Enter the lost_reason value: 0
Enter the Agent_id value: c213697430c006013012dd2aca82dd9732aa0a1a6bca1371c92f18fd11d9bc5f
Enter the source value: 7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2b7c51be879f0c37ca8
Enter the source_city value: 9b8cc3c63cdf447e463c11544924bf027945cbd29675f77955bb36364356c14e
Enter the source_country value: e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a9750aa698e797e9c96
Enter the utm_source value: bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07ee6acc68762d12c2e3
Enter the utm_medium value: 09076eb7665d1fb9389c7c4517fee0b00e43092eb34821b09b5730c41ebcc50c
Enter the des_city value: 11ab03a1a8c367191355c152f39fe28cae5e426fce49efb320230ca4ae3f97a1
Enter the des_country value: 8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512ff098eaa56a4d5fb19
Enter the room_type value: Ensuite
the expected lead score is : [51.04712543]
We won customer


In [312]:
print("Agent",df['Agent_id'].iloc[0])
print("source",df['source'].iloc[0])
print("source_city",df['source_city'].iloc[0])
print("source_country",df['source_country'].iloc[0])
print("utm_source",df['utm_source'].iloc[0])
print("utm_medium",df['utm_medium'].iloc[0])
print("des_city",df['des_city'].iloc[0])
print("room_type",df['room_type'].iloc[0])
print("room_type",df['room_type'].iloc[0])
print("des_country",df['des_country'].iloc[0])

Agent c213697430c006013012dd2aca82dd9732aa0a1a6bca1371c92f18fd11d9bc5f
source 7aae3e886e89fc1187a5c47d6cea1c22998ee610ade1f2b7c51be879f0c37ca8
source_city 9b8cc3c63cdf447e463c11544924bf027945cbd29675f77955bb36364356c14e
source_country e09e10e67812e9d236ad900e5d46b4308fc62f5d69446a9750aa698e797e9c96
utm_source bbdefa2950f49882f295b1285d4fa9dec45fc4144bfb07ee6acc68762d12c2e3
utm_medium 09076eb7665d1fb9389c7c4517fee0b00e43092eb34821b09b5730c41ebcc50c
des_city 11ab03a1a8c367191355c152f39fe28cae5e426fce49efb320230ca4ae3f97a1
room_type Ensuite
room_type Ensuite
des_country 8d23a6e37e0a6431a8f1b43a91026dcff51170a89a6512ff098eaa56a4d5fb19


In [315]:
final_results = pd.DataFrame(final_dict)

In [319]:
#final_results.to_csv("final_result.csv")

In [318]:
final_results

,Feature,Score,Model_name,rmse_val,r2_score,mse,mae,f1,precision,recall
0,budget_new,"[0.001925721320453544, 0.0038134492074047843]",Simple_Linear,0.265456,0.003813,0.070467,0.137974,na,na,na
1,lease_span_avg_week,"[0.010465134454557101, 0.015019078941020614]",Simple_Linear,0.263959,0.015019,0.069674,0.136337,na,na,na
2,Agent_id_encoded,"[0.0007160696718152559, -1.7145324924916494e-05]",Simple_Linear,0.265966,-0.000017,0.070738,0.138310,na,na,na
3,lost_reason_encoded,"[0.40353047597778713, 0.41667520698371463]",Simple_Linear,0.203131,0.416675,0.041262,0.132742,na,na,na
4,source_encoded,"[0.001320461103237025, 0.0015718306123276848]",Simple_Linear,0.265754,0.001572,0.070625,0.138332,na,na,na
...,...,...,...,...,...,...,...,...,...,...
118,source_encoded,"[0.7977733410996394, 0.8007639990098356]",SVM,0.456161,0.800764,0.208082,0.157476,na,na,na
119,source_city_encoded,"[0.7917244068095224, 0.7934945791912803]",SVM,0.464408,0.793495,0.215675,0.154953,na,na,na
120,des_city_encoded,"[0.7949188311458169, 0.7973759157904106]",SVM,0.460023,0.797376,0.211621,0.155127,na,na,na
121,des_country_encoded,"[0.8024807339956747, 0.8048105865058437]",SVM,0.451504,0.804811,0.203856,0.155833,na,na,na


# **Project Completed by - Swetanshu Pandey**

**UniAcco**